In [1]:
# NCb: PNB
NCb = [(30.898499, 75.872277), (30.898498, 75.872236), (30.898504, 75.872180), (30.898514, 75.872140), (30.898529, 75.872094),
        (30.898548, 75.872043)]

SCb = [(30.898266, 75.872104), (30.898301, 75.872072), (30.898335, 75.872029), (30.898362, 75.871992), 
       (30.898390, 75.871954), (30.898417, 75.871918)]

NM = [(30.898405, 75.872246), (30.898414, 75.872198), (30.898425, 75.872136), (30.898443, 75.872086),
      (30.898459, 75.872046), (30.898476, 75.872005)]

SM = [(30.898325, 75.872164), (30.898373, 75.872115), (30.898417, 75.872058), (30.898457, 75.872003), (30.898486, 75.871953),
      (30.898505, 75.871917)]

NC = [(30.898450, 75.872252), (30.898458, 75.872201), (30.898473, 75.872147), (30.898488, 75.872098), 
      (30.898503, 75.872053), (30.898520, 75.872004) ]

SC = [(30.898300, 75.872135), (30.898334, 75.872094), (30.898373, 75.872049), (30.898405, 75.871999), (30.898427, 75.871970),
      (30.898449, 75.871935)]

In [2]:
def loc(pedestrian_id):
    import pandas as pd
    import numpy as np
    from shapely.geometry import Point, LineString

    # Define the BRCb, BLCb, BLM, BRM, BRC, and BLC lines as shapely LineString objects
    BRCb_line = LineString(NCb)
    BLCb_line = LineString(SCb)
    BLM_line = LineString(NM)
    BRM_line = LineString(SM)
    BRC_line = LineString(NC)
    BLC_line = LineString(SC)

    # Select a pedestrian trajectory
    pedestrian_df = df[(df['ID'] == pedestrian_id) & (df['Class'] == ' Pedestrian')]

    # Calculate the distance between each point in the pedestrian's trajectory and each of the lines
    trajectory_points = np.array(pedestrian_df[['X', 'Y']])
    line_distances = np.zeros((len(trajectory_points), 6))
    for i, point in enumerate(trajectory_points):
        point = Point(point)
        line_distances[i, 0] = point.distance(BRCb_line)
        line_distances[i, 1] = point.distance(BLCb_line)
        line_distances[i, 2] = point.distance(BLM_line)
        line_distances[i, 3] = point.distance(BRM_line)
        line_distances[i, 4] = point.distance(BRC_line)
        line_distances[i, 5] = point.distance(BLC_line)

    # Assign each point to the line with the minimum distance
    line_names = ['NCb', 'SCb', 'NM', 'SM', 'NC', 'SC']
    nearest_lines = np.argmin(line_distances, axis=1)
    pedestrian_df['Nearest Line'] = [line_names[i] for i in nearest_lines]
    return (pedestrian_df[['frame','ID' ,'X', 'Y', 'Nearest Line']])

In [3]:
def dirn(pedestrian_id):
    import pandas as pd
    from scipy.spatial.distance import cdist

    # Filter the dataframe to only include the trajectory of the pedestrian of interest
    pedestrian_df = df[(df['ID'] == pedestrian_id) & (df['Class'] == ' Pedestrian')]

    # Find the closest point on the BRCb line
    closest_brcb = min(NCb, key=lambda p: cdist([p], pedestrian_df[['X', 'Y']]).min())

    # Find the closest point on the BLCb line
    closest_blcb = min(SCb, key=lambda p: cdist([p], pedestrian_df[['X', 'Y']]).min())

    # Determine which point is closer to the start of the trajectory
    start_point = pedestrian_df[['X', 'Y']].iloc[0].values
    if cdist([start_point], [closest_brcb]).min() < cdist([start_point], [closest_blcb]).min():
        return('NCb to SCb')
    else:
        return('SCb to NCb')